In [ ]:
%cd /content
!apt install -qqy
!pip install torchsde einops pyaml diffusers transformers peft accelerate aiohttp kornia spandrel sentencepiece ipywidgets termcolor moviepy==1.0.3

!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/ComfyUI-Manager

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/clip -o t5xxl_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Lightricks/LTX-Video/resolve/main/ltx-video-2b-v0.9.safetensors -d /content/ComfyUI/models/checkpoints -o ltx-video-2b-v0.9.safetensors

In [1]:
%env CUDA_VISIBLE_DEVICES=1
%cd /content/ComfyUI

import os, shutil, json, requests, random, time
from urllib.parse import urlsplit

import torch
from PIL import Image
from moviepy.editor import ImageSequenceClip
import numpy as np

from nodes import NODE_CLASS_MAPPINGS
import comfy_extras
from comfy_extras import  nodes_images, nodes_lt, nodes_custom_sampler

CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
LTXVConditioning = nodes_lt.NODE_CLASS_MAPPINGS["LTXVConditioning"]()
SamplerCustom = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustom"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
LTXVScheduler = nodes_lt.NODE_CLASS_MAPPINGS["LTXVScheduler"]()
EmptyLTXVLatentVideo = nodes_lt.NODE_CLASS_MAPPINGS["EmptyLTXVLatentVideo"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
SaveAnimatedWEBP = nodes_images.NODE_CLASS_MAPPINGS["SaveAnimatedWEBP"]()

with torch.inference_mode():
    clip = CLIPLoader.load_clip("t5xxl_fp16.safetensors", type="ltxv")[0]
    model, _, vae = CheckpointLoaderSimple.load_checkpoint("ltx-video-2b-v0.9.safetensors")

def download_file(url, save_dir, file_name):
    os.makedirs(save_dir, exist_ok=True)
    file_suffix = os.path.splitext(urlsplit(url).path)[1]
    file_name_with_suffix = file_name + file_suffix
    file_path = os.path.join(save_dir, file_name_with_suffix)
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, 'wb') as file:
        file.write(response.content)
    return file_path

def webp_to_mp4(input_webp, output_mp4, fps=10):
    with Image.open(input_webp) as img:
        frames = []
        try:
            while True:
                frame = img.copy()
                frames.append(frame)
                img.seek(img.tell() + 1)
        except EOFError:
            pass
    temp_dir = "temp_frames"
    os.makedirs(temp_dir, exist_ok=True)
    temp_frame_paths = []
    for i, frame in enumerate(frames):
        frame_path = os.path.join(temp_dir, f"frame_{i:04d}.png")
        frame.save(frame_path)
        temp_frame_paths.append(frame_path)
    clip = ImageSequenceClip(temp_frame_paths, fps=fps)
    clip.write_videofile(output_mp4, codec="libx264", fps=fps)
    for path in temp_frame_paths:
        os.remove(path)
    os.rmdir(temp_dir)

@torch.inference_mode()
def generate(input):
    values = input["input"]

    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    frame_rate = values['frame_rate']
    sampler_name = values['sampler_name']
    steps = values['steps']
    max_shift = values['max_shift']
    base_shift = values['base_shift']
    stretch = values['stretch']
    terminal = values['terminal']
    width = values['width']
    height = values['height']
    length = values['length']
    add_noise = values['add_noise']
    noise_seed = values['noise_seed']
    cfg = values['cfg']
    fps = values['fps']

    if noise_seed == 0:
        random.seed(int(time.time()))
        noise_seed = random.randint(0, 18446744073709551615)
    print(noise_seed)

    conditioning_positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    conditioning_negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    positive, negative = LTXVConditioning.append(conditioning_positive, conditioning_negative, frame_rate)
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = LTXVScheduler.get_sigmas(steps, max_shift, base_shift, stretch, terminal, latent=None)[0]
    latent_image = EmptyLTXVLatentVideo.generate(width, height, length, batch_size=1)[0]
    samples = SamplerCustom.sample(model, add_noise, noise_seed, cfg, positive, negative, sampler, sigmas, latent_image)[0]
    images = VAEDecode.decode(vae, samples)[0].detach()
    video = SaveAnimatedWEBP.save_images(images, fps, filename_prefix=f"ltx-video-{noise_seed}-tost", lossless=False, quality=90, method="default")
    source = video['ui']['images'][0]['filename']
    destination = f"/content/ltx-video-{noise_seed}-tost.webp"
    shutil.move(f"/content/ComfyUI/output/{source}", destination)
    webp_to_mp4(f"/content/ltx-video-{noise_seed}-tost.webp", f"/content/ltx-video-{noise_seed}-tost.mp4", fps=fps)
    
    result = f"/content/ltx-video-{noise_seed}-tost.mp4"

    return result

env: CUDA_VISIBLE_DEVICES=1
/content/ComfyUI


/content/ComfyUI-venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
input = { 
        "input": {
            "positive_prompt": "A woman with long brown hair and light skin smiles at another woman with long blonde hair. The woman with brown hair wears a black jacket and has a small, barely noticeable mole on her right cheek. The camera angle is a close-up, focused on the woman with brown hair's face. The lighting is warm and natural, likely from the setting sun, casting a soft glow on the scene. The scene appears to be real-life footage.",
            "negative_prompt": "low quality, worst quality, deformed, distorted, disfigured, motion smear, motion artifacts, fused fingers, bad anatomy, weird hand, ugly",
            "frame_rate": 25,
            "sampler_name": "euler",
            "steps": 30,
            "max_shift": 2.05,
            "base_shift": 0.95,
            "stretch": True,
            "terminal": 0.10,
            "width": 768,
            "height": 512,
            "length": 97,
            "add_noise": True,
            "noise_seed": 0,
            "cfg": 3.0,
            "fps": 24
    }
}
video = generate(input)
from IPython.display import Video
Video(video, embed=True)